### 1 - Read in data
We need the expression values for all cells of all genes in the scRNA-seq dataset. We also need to know which genes have been used in the spatial reference expression dataset. In the following we read in those two datasets.

In [2]:
base_dir <- "/home/jovyan/storage"

# path to scRNA-seq DGE
dge_path <- paste0(base_dir, "/data/dge_day4_Oct-Nofilter.csv")

# path to spatial reference expression matrix
coord_path <- paste0(base_dir, "/data/confocal_states0-FilterWUSCLVtop100.csv")

# path to predict 3D gene expression profiles
sdge_path <- paste0(base_dir, "/data/sdge.csv")

# Load scRNA-seq dataset
dge <- read.csv(file = dge_path, row.names=1)
dge <- as.data.frame(t(dge))

# Load spatial expression reference dataset
ref <- as.matrix(read.csv(file = coord_path, row.names = 1, sep=","))

# Load predicted 3D gene expression profiles
sdge <- read.csv(file = sdge_path, row.names = 1)

### 2 - Calculate PEP-scores for genes in scRNA-seq datatset

First we obtain a correlation matrix that tells us how much each gene in the scRNA-seq dataset correlates with the genes in the spatial expression reference dataset. The resulting matrix has the dimension (scRNA-seq genes x reference genes).

In [3]:
genes <- intersect(colnames(dge), colnames(ref))
co <- cor(x = dge, y = dge[, genes], method = 'spearman')
dim(co)

[1] 19718    23

Next we find the reference gene that has the highest correlation for all genes in the scRNA-seq dataset and record the correlation value. This value represents the PEP-score.  
The resulting data.frame contains the PEP-score for each gene in the scRNA-seq dataset as well as the reference gene that the correlation is based on.

In [4]:
# get top1 correlation for all genes with ref. genes
cor_top1 <- NULL
for(gen in rownames(co)){
  if(gen %in% colnames(co)){
      ord <- order(co[gen,], decreasing = TRUE)[2]
  } else {
      ord <- order(co[gen,], decreasing = TRUE)[1]
  }
  row <- c(as.numeric(co[gen, ord]), colnames(co)[ord])
  cor_top1 <- rbind(cor_top1, row)
}
cor_top1_df <- as.data.frame(cor_top1)

rownames(cor_top1_df) <- rownames(co); colnames(cor_top1_df) <- c('PEP-score', 'top_cor_gene')
cor_top1_df$`PEP-score` <- round(as.numeric(cor_top1_df$`PEP-score`),2)
cor_top1_df[1:4,]

,PEP-score,top_cor_gene
,<dbl>,<chr>
AT1G01010,0.04,AT1G62360
AT1G01020,0.03,AT4G36920
AT1G01030,0.09,AT1G69120
AT1G01040,0.04,AT1G62360


### 3 - Interpretation of the PEP-score
The PEP-score can be used as an approximation on how confident we are that the model we trained in the notebook "1 - Predict_3D_gene_expression_profiles.ipynb" can give us (biologically) reasonable results. Since the score is only based on the expression of genes in the scRNA-seq and not the parameters that we used for the predictions, the PEP-score can only be used as a rough approximation for the out-of-sample prediction performance.  
The higher the PEP-score for a de novo 3D gene expression prediction, the more confident we are that the prediction makes sense biologically.

### 4 - Connection to the AUCROC
The prediction performance for..

In [6]:
install.packages("pROC")

Installing package into ‘/home/jovyan/R/x86_64-pc-linux-gnu-library/4.2’
(as ‘lib’ is unspecified)



In [7]:
library(pROC)

Type 'citation("pROC")' for a citation.


Attaching package: ‘pROC’


The following objects are masked from ‘package:stats’:

    cov, smooth, var




In [16]:
calculate_AUC <- function(type="roc", return_type = "auc", pre, ref){
      defaultW <- getOption("warn") 
      options(warn = -1) 

      auc_list <- NULL
      genes_tair <- intersect(colnames(ref), colnames(pre))
      for(gene_tair in genes_tair){
        if(type == "roc"){
            suppressMessages(
              aucroc <- roc(as.numeric(ref[, gene_tair]), as.numeric(pre[, gene_tair]), plot=FALSE, legacy.axes=TRUE, direction = "<")
            )
          auc_list[[gene_tair]] <- aucroc$auc[1]
        }else if(type == "pr"){
            suppressMessages(
              aucpr <- cvAUC::AUC(predictions=as.numeric(pre[, gene_tair]), labels=as.numeric(ref[, gene_tair]))
            )
          auc_list[[gene_symbol]] <- aucpr
        }
      }
    options(warn = defaultW)
    return(auc_list)
}

In [10]:
sdge[1:4,1:4]

,x,y,z,AT1G01010
,<dbl>,<dbl>,<dbl>,<dbl>
0,90.261,67.545,5.049,2.381
1,58.430,97.106,5.191,2.432
2,47.912,72.668,5.500,2.279
3,58.578,88.082,5.723,2.418


In [18]:
auc <- calculate_AUC(pre = sdge, ref = ref)
auc <- data.frame(auc)
auc <- as.data.frame(t(auc))

In [19]:
auc

,V1
,<dbl>
x,1.0000000
y,1.0000000
z,1.0000000
AT1G19850,1.0000000
AT1G24260,1.0000000
AT1G30490,1.0000000
AT1G62360,1.0000000
AT1G69120,1.0000000
AT1G76420,1.0000000
